## Import Library

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob, glob2
import albumentations as A
import time
import random
import os
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
!nvidia-smi

## Prepare dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Shareddrives/thinh5/capstone-project/data/zip

In [ ]:
!cp pushups.zip /content/

In [ ]:
!unzip /content/pushups.zip -d /content/dataset

In [ ]:
!rm /content/pushups.zip

In [ ]:
TYPE = 'up'
# TYPE = 'down'

In [ ]:
class_names = ['right', 'wrong']
num_classes = 2

In [ ]:
dataset_folder = '/content/dataset/pushups/horizontal'
save_path = '/content/drive/MyDrive/Shareddrives/thinh5/capstone-project/models/pushups'
img_size = (224, 224)

In [ ]:
all_files = glob.glob(f'{dataset_folder}/*/{TYPE}/*/*[.png|.jpg]')

In [ ]:
len(all_files)

In [ ]:
for cls_name in class_names:
  print(cls_name, len(glob.glob(f'{dataset_folder}/*/{TYPE}/{cls_name}/*[.png|.jpg]')))

In [ ]:
for file_name in all_files:
  if 'Copy' in file_name:
    org = file_name.replace(' - Copy', '')
    if org in all_files:
      print(org)

## Augmentation

In [ ]:
transform = A.Compose([
    
        A.HorizontalFlip(p=0.5),
        # A.VerticalFlip(p=0.5),
        
        A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.2, rotate_limit=5, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.5),
        A.RandomSunFlare(flare_roi=(0, 0, 1, 1), src_radius=50, p=0.1),
        A.IAAPerspective(scale=(0.01, 0.01), p=0.1),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
            A.Blur(blur_limit=3, p=0.3),
            A.MedianBlur(blur_limit=3, p=0.3),
        ], p=0.3),
        A.CLAHE(clip_limit=1.5, p=0.3),
        
        A.RandomCrop(width=1020, height=764, p=0.5),
    ])

In [ ]:
fig = plt.figure(figsize=(20, 20))
columns = 4
rows = 4
for i in range(1, columns*rows +1):
    test_img_path = random.choice(all_files)
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    transformed = transform(image=img)
    img = transformed["image"]
    # img = (( X[i-1].copy() + 1)*127.5).astype('uint8')
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

## Data Loader

In [ ]:
class DataGenerator(Sequence):
    def __init__(self,
                img_paths,
                class_names,
                batch_size=128,
                img_size=(224,224),
                n_channels=3,
                shuffle=True,
                augmentations=None,
                ):
        self.img_paths = img_paths
        self.class_names = np.array(class_names) 
        self.batch_size = batch_size
        self.img_size = img_size
        self.n_channels = n_channels 
        self.n_classes = len(class_names)
        self.shuffle = shuffle 
        self.augmentations = augmentations
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.img_paths) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        batch_img_paths = [self.img_paths[k] for k in indexes]
        
        # Generate data
        X, y = self.__data_generation(batch_img_paths)
        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.img_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, batch_img_paths):
        X = np.empty((self.batch_size, *self.img_size, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)
        
        try: 
            for i, img_path in enumerate(batch_img_paths):
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
                # img = cv2.resize(img, self.img_size)
                
                if self.augmentations is not None:
                    # Augment an image
                    transformed = self.augmentations(image=img)
                    img = transformed["image"]
                    
                img = cv2.resize(img, self.img_size, interpolation=cv2.INTER_AREA)
                label = img_path.split('/')[-2]
                label = (self.class_names == label)*1

                X[i] = img*1.0
                y[i] = label
        except:
            print(img_path)
        # Normalize batch data
        X /= 127.5
        X -= 1.

        return X, y

In [ ]:
# ## If not sure model works fine, should use val set
X_train, X_test, _, _ = train_test_split(all_files, all_files, test_size=0.2, random_state=42)
print(len(X_train))

train_generator = DataGenerator(X_train, class_names, batch_size=32, augmentations=transform)
test_generator = DataGenerator(X_test, class_names, batch_size=32, augmentations=transform)
print(len(train_generator), len(test_generator))

In [ ]:
# train_generator = DataGenerator(all_files, class_names, batch_size=32, augmentations=transform)
# val_generator = DataGenerator(X_test, class_names)
# print(train_generator.__len__())

In [ ]:
### Check generator
train_generator.on_epoch_end()
X, y = train_generator.__getitem__(1)

In [ ]:
fig = plt.figure(figsize=(20, 20))
columns = 4
rows = 4
for i in range(1, columns*rows +1):
    img = (( X[i-1].copy() + 1)*127.5).astype('uint8')
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

## Prepare Model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = (*img_size, 3), include_top = False, weights = "imagenet")
base_model.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(num_classes)(x)
outputs = tf.keras.layers.Activation('softmax')(x)
model =  tf.keras.Model(base_model.input, outputs)

model.summary()

In [ ]:
### Should use 'val_accuracy' if not sure that model works fine
METRICS =['accuracy',
            keras.metrics.Precision(name="precision"),
            keras.metrics.Recall(name="recall"),
            keras.metrics.AUC(name="auc"),]
early_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

checkpoint_filepath = save_path + '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy', 
    mode='max',
    save_best_only=True)

## Train

### Warm up model with hight LR + free base model

In [ ]:
learning_rate = 0.001
num_epochs = 5
base_model.trainable = False
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=METRICS
)

In [ ]:
# history = model.fit(train_generator, epochs=num_epochs, callbacks=[early_callback, model_checkpoint_callback])
history = model.fit(train_generator, validation_data=test_generator, epochs=num_epochs, callbacks=[early_callback, model_checkpoint_callback])

### Fine tune model with low learning rate + unfreeze base model

In [ ]:
num_epochs = 40
base_model.trainable = True
# extractor.trainable = True

initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=70,
    decay_rate=0.93,
    staircase=True)
# boundaries = [700]
# values = [1e-6, 5e-7]
# learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
#     boundaries, values)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=METRICS
)
model.summary()

In [ ]:
# history = model.fit(train_generator, epochs=num_epochs, callbacks=[early_callback, model_checkpoint_callback])
history = model.fit(train_generator, validation_data=test_generator, epochs=num_epochs, callbacks=[model_checkpoint_callback])

### Save trained model

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m%Y_%H%M%S")
print("date and time =", dt_string)
model_save_path = f'{save_path}/best_mobinet_{TYPE}_{dt_string}'

model.save(model_save_path)
print(f'Model saved at {model_save_path}')

## Test Model

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
model_save_path = "/content/drive/MyDrive/Shareddrives/thinh5/capstone-project/models/pushups-down/finetune/MobileNetV2_0.001_0.0001"

In [ ]:
### Put the path of saved model
print ("load model ....")
model = keras.models.load_model(model_save_path)
print ("load model successfully!")
img_size = 224

In [ ]:
# !pip install wandb

In [ ]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('xuannhamng28/squats3-up/mobilenet:v5', type='model')
# artifact_dir = artifact.download()
# model = keras.models.load_model(artifact_dir)

In [ ]:
TYPE = 'down'
img_size = 224
class_names = ['right', 'wrong']
num_classes = 2

In [ ]:
test_folder = "/content/drive/MyDrive/Shareddrives/thinh5/capstone-project/data/test_dataset/pushups/horizontal"
test_img_list = glob.glob(f'{test_folder}/*/{TYPE}/*[right|wrong]/*[.jpg|.png]')
len(test_img_list)

In [ ]:
def classify(img_name, show=True):
    
    input = cv2.imread(img_name)[...,::-1]
    input = cv2.resize(input,(img_size, img_size), interpolation=cv2.INTER_AREA)
    input = np.reshape(input,[1,224,224,3])*1.0
    # input = keras.applications.mobilenet_v2.preprocess_input(input)
    input /= 127.5
    input -= 1.
    out = model.predict(input)[0]
    print(out)
    if show:
      print(img_name)
      cv2_imshow(cv2.imread(img_name))
    return class_names[np.argmax(out)],np.round(out[np.argmax(out)],4)

In [ ]:
classify(test_img_list[7])

In [ ]:
total, count = 0, 0
for img_path in test_img_list:
  print(img_path)
  label = img_path.split('/')[-2]
  pred_lb, pred_conf = classify(img_path, show=False)
  if pred_lb != label:
    count += 1
    cv2_imshow(cv2.resize(cv2.imread(img_path), (img_size, img_size)))


In [ ]:
total, count, len(test_img_list)

##  Convert model to tflite

In [ ]:
saved_model_name = 'MobileNetV2_down'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


# Save the model. 
tflite_save_path = f"/content/drive/MyDrive/Shareddrives/thinh5/capstone-project/models/tflite/{saved_model_name}.tflite"
print(tflite_save_path)
with open(tflite_save_path, 'wb') as f:
  f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

In [ ]:
test_img_path = test_img_list[7]
test_img_path

In [ ]:
input = cv2.imread(test_img_path)
input = cv2.resize(input, dsize=(224, 224), interpolation=cv2.INTER_AREA)
input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
input = (input/127.5) - 1
input = input.reshape(1, 224, 224, 3)

In [ ]:
input_data = tf.cast(input,tf.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])[0]
print(output_data)

In [ ]:
class_idx = tf.argmax(output_data, axis=0)
class_idx